## Выбор модели для задач классификации

Для задач классификации существует несколько способов выбора модели:

1. **Обучить маленькую модель с нуля**: Выбор легковесной архитектуры модели и полное обучение её на нашем наборе данных.
2. **Fine-Tuning предобученной модели**: Используем модель, которая была предобучена авторами на другом наборе данных, и только последние слои переобучается на нашем конкретном наборе данных.
3. **Использовать предобученные веса напрямую**: В этом методе используется предобученная модель без дополнительного обучения.

Лучшим вариантом является Fine-Tuning, поэтому в качестве бейзлайна возьмем маленькую (5.3M) EfficientNet B0 с претрейновыми параметрами

[Оригинальная статья](https://arxiv.org/abs/1905.11946)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import os
import zipfile
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Загрузка гиперпараметров из JSON (создайте папку classification, если её еще нет в корне диска Google Drive)
with open('/content/drive/MyDrive/classification/hyperparams.json', "r") as f:
  hyperparams = json.load(f)

In [ ]:
# Загрузка названий классов из JSON
with open("/content/drive/MyDrive/classification/class_to_idx.json", "r")  as f:
  class_to_idx = json.load(f)


**Структура** c*lassification_dataset.zip*: (внутри папка с таким же названием)

```
classification_dataset/
├── train/
│   ├── class_name1/
│   └── class_name2/
│   └── ...

└── test/
    ├── class_name1/
    └── class_name2/
    └── ...
```



In [ ]:
local_zip = '/content/drive/MyDrive/classification/classification_dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/dataset')
zip_ref.close()

In [ ]:
# Класс для аугментации данных
class AugmentedDataset(Dataset):
    def __init__(self, original_folder, target_size=50, transform=None):
        self.transform = transform
        self.samples = []
        self.class_to_idx = class_to_idx

        # Проверяем, что все классы из JSON есть в папке
        class_names = os.listdir(original_folder)
        missing_classes = [cls for cls in class_to_idx.keys() if cls not in class_names]
        if missing_classes:
            raise ValueError(f"Классы {missing_classes} из class_to_idx.json отсутствуют в папке {original_folder}")

        # Собираем пути к изображениям для каждого класса
        for class_name in class_names: # Используем отсортированные имена классов
            class_path = os.path.join(original_folder, class_name)
            images = [os.path.join(class_path, img) for img in os.listdir(class_path)]
            # Повторяем изображения до достижения целевого размера
            for i in range(target_size):
                self.samples.append((images[i % len(images)], class_name))

    # Возвращает общее количество элементов в наборе данных
    def __len__(self):
        return len(self.samples)

    # Возвращает одно изображение и его метку по индексу
    def __getitem__(self, idx):
        img_path, class_name = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        # Применяем аугментации
        if self.transform:
            image = self.transform(image)

        # Преобразуем метку класса в числовой формат
        label = self.class_to_idx[class_name]  # Преобразуем имя класса в индекс
        return image, label  # Оставляем числовую метку для обучения

In [ ]:
# Класс для создания датасета под Test выборку
class TestDataset(Dataset):
    def __init__(self, original_folder, transform=None):
        self.transform = transform
        self.samples = []
        self.class_to_idx = class_to_idx  # Загружаем метки классов из JSON

        # Проверяем, что все классы из JSON есть в папке
        class_names = os.listdir(original_folder)
        missing_classes = [cls for cls in class_to_idx.keys() if cls not in class_names]
        if missing_classes:
            raise ValueError(f"Классы {missing_classes} из class_to_idx.json отсутствуют в {original_folder}")

        # Собираем пути к изображениям
        for class_name in class_names:
            class_path = os.path.join(original_folder, class_name)
            images = [os.path.join(class_path, img) for img in os.listdir(class_path)]
            for img_path in images:
                self.samples.append((img_path, class_name))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, class_name = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        # Применяем трансформации (без аугментаций)
        if self.transform:
            image = self.transform(image)

        # Преобразуем имя класса в индекс
        label = self.class_to_idx[class_name]
        return image, label

In [ ]:
# Трансформы с аугментациями для тренировочных данных
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Трансформы для тестовых данных
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Создание аугментированных датасетов
train_dataset = AugmentedDataset(
    original_folder='/content/dataset/classification_dataset/train',
    target_size=50,
    transform=train_transform
)

test_dataset = TestDataset(
    original_folder='/content/dataset/classification_dataset/test',
    transform=test_transform
)


# DataLoader'ы
train_loader = DataLoader(
    train_dataset,
    batch_size=hyperparams['batch_size'],
    shuffle=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=hyperparams['batch_size'],
    shuffle=False
)

In [ ]:
# Проверим классы
test_dataset.class_to_idx

In [ ]:
# Создание обратного словаря idx_to_class
idx_to_class = {v: k for k, v in class_to_idx.items()}

In [ ]:
# Проверим классы
print(train_dataset.class_to_idx)

In [ ]:
# Загрузка модели EfficientNet
model = efficientnet_b0(EfficientNet_B0_Weights.DEFAULT)

# Заморозка всех слоев, кроме последнего
for param in model.parameters():
    param.requires_grad = False

# Замена финального классификатора
num_features = model.classifier[1].in_features
num_classes = len(class_to_idx)  # Количество классов из JSON
model.classifier = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(),
    nn.Linear(128, num_classes),  # Многоклассовая классификация
    # На выходе оставляем сырые логиты, так как nn.CrossEntropyLoss() их ожидает на вход
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()  # Для многоклассовой классификации
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=hyperparams['learning_rate'],
    weight_decay=hyperparams['weight_decay']
)

In [ ]:
# Обучение модели
for epoch in range(hyperparams['num_epochs']):
    model.train()
    train_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    # Валидация
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)  # Получаем индекс максимального значения
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    # Вывод статистики
    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(test_loader.dataset)
    accuracy = correct / total

    print(f"Epoch {epoch+1}/{hyperparams['num_epochs']}")
    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}")

In [ ]:
# Сохранение модели
torch.save(model, '/content/trained_model_classification.pt')

# Копирование на Google Drive
!cp "/content/trained_model_classification.pt" "/content/drive/MyDrive/classification/trained_model_classification.pt"
print("Модель сохранена")